In [1]:
# AutoDL官方学术资源加速
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [2]:
import sys
import os

# 添加项目根目录到Python路径
project_root = "/home/cuipeng/Gemma"
sys.path.append(project_root)

# 现在可以正常导入src下的模块
from src.core.model.model_initializer import initialize_model_and_tokenizer
from src.core.utils.model_utils import generate_response, apply_chat_template, format_user

In [3]:
# Least-to-Most Prompting (L2M)

import torch # type: ignore
from transformers import AutoTokenizer, AutoModelForCausalLM # type: ignore
from transformers import BitsAndBytesConfig # type: ignore
from peft import PeftModel  # type: ignore # 导入PeftModel用于加载微调模型

In [4]:
class L2MSolver:
    def __init__(self, model, tokenizer):
        """
        初始化L2M求解器
        Args:
            model: 已初始化的模型实例
            tokenizer: 已初始化的tokenizer实例
        """
        self.model = model
        self.tokenizer = tokenizer
        
        # 添加 few-shot 示例
        self.decompose_examples = """
        问题1：量子纠缠现象是什么，它有什么应用？

        分解步骤：
        1. 什么是量子态和量子叠加？
        2. 量子纠缠的基本概念是什么？
        3. 量子纠缠现象如何被实验证实？
        4. 量子纠缠在量子通信和量子计算中有什么应用？

        问题2：区块链技术如何确保交易安全？

        分解步骤：
        1. 什么是哈希函数和加密算法？
        2. 区块链的基本结构是什么？
        3. 区块链如何实现去中心化验证？
        4. 为什么区块链的交易记录难以篡改？
        """

        self.decompose_system_prompt = """
        你是一位擅长分析和分解复杂问题的助手。你的主要任务是将复杂问题分解成若干个由简单到复杂的子问题。

        在分解问题时，你应该：
        1. 仔细分析问题的各个组成部分
        2. 识别解决问题所需的关键知识和技能
        3. 确保子问题之间具有逻辑连贯性
        4. 确保每个子问题的答案都能为最终问题的解决提供必要的支持

        你应该遵循以下原则：
        1. 子问题数量应保持在3-5个之间
        2. 子问题应该按照难度递增的顺序排列
        3. 每个子问题都应该明确、具体且可回答
        4. 避免输出无关的解释或评论

        输出格式要求：
        1. 必须严格按照"分解步骤："开头
        2. 每个子问题前标注序号
        3. 除了分解的子问题外，不要包含任何其他内容

        示例输出：
        分解步骤：
        1. [简单的子问题]
        2. [稍复杂的子问题]
        3. [较复杂的子问题]
        4. [最复杂的子问题]

        请记住：你的回答应该简洁明了，只包含问题分解本身，不需要任何额外的解释或评论。
        """

    def decompose_question(self, complex_question):
        dialogue = [
            {
                "role": "system",
                "content": self.decompose_system_prompt
            },
            {
                "role": "user",
                "content": f"""请参考以下示例，将复杂问题分解为子问题：

    {self.decompose_examples}

    现在请分解这个问题：{complex_question}

    请严格按照示例格式输出，以"分解步骤："开头。"""
            }
        ]
        
        prompt = apply_chat_template(dialogue)
        response = generate_response(self.model, self.tokenizer, prompt, max_new_tokens=512, temperature=0.3) # 降低温度，以保持回答的一致性
        
        # 改进子问题提取方法 ########################################################
        def extract_questions(text):
            import re
            
            pattern = r'\d+\.\s*([^\n]+)' # 匹配：1. This is the first line. 捕获组：This is the first line.
            questions = re.findall(pattern, text)
            
            if questions: # 如果question不为空，则表示提取成功
                return questions
            
            # 如果第一次提取失败，则尝试从"分解步骤："开始提取
            if "分解步骤：" in text:
                text = text.split("分解步骤：")[1].strip()
            
            lines = text.split('\n')
            questions = []
            
            for line in lines:
                line = line.strip()
                line = re.sub(r'^\d+\.\s*', '', line)
                if line and not line.startswith('分解步骤') and not line.startswith('要求'):
                    questions.append(line)
            
            return questions
        ########################################################################
        
        sub_questions = extract_questions(response)
        
        # 边缘处理
        if not sub_questions:
            backup_dialogue = [
                {"role": "system", "content": self.decompose_system_prompt},
                {"role": "user", "content": f"""之前的回答格式不正确。请重新分解问题，严格按照以下格式输出：
                 问题：{complex_question}

                 1. [第一个子问题]
                 2. [第二个子问题]
                 3. [第三个子问题]
                 ...

                请只输出编号和问题，不要有其他内容。"""}
                ]
            backup_prompt = apply_chat_template(backup_dialogue)
            
            response = generate_response(self.model, self.tokenizer, backup_prompt)
            sub_questions = extract_questions(response)
        
        # 如果还没有sub_questions，则报错
        if not sub_questions:
            raise ValueError("无法正确分解问题，请检查模型输出格式")
            
        return sub_questions

    def solve_with_context(self, question, context=""):
        dialogue = [
            {
                "role": "system",
                "content": "你是一个专业的问题解答专家。请参考示例风格，基于已知信息回答问题。"
            },
            {
                "role": "user",
                "content": f"""示例：
    问题：什么是量子态？
    已知信息：无
    答案：量子态是描述量子系统状态的数学表达，它可以同时存在于多个基本状态的叠加中。这种特性使得量子系统具有独特的性质。

    现在请回答：
    已知信息：
    {context}

    问题：{question}"""
            }
        ]
        
        prompt = apply_chat_template(dialogue)
        return generate_response(self.model, self.tokenizer, prompt, max_new_tokens=1024, temperature=0.7)

    def solve(self, complex_question):
        # 1. 分解问题
        sub_questions = self.decompose_question(complex_question) # 返回一个list
        
        # 2. 逐步解决
        context = ""
        solutions = []
        
        for q in sub_questions:
            solution = self.solve_with_context(q, context)
            solutions.append({"question": q, "solution": solution})
            context += f"\n问题：{q}\n答案：{solution}\n"
        
        # 3. 最终整合答案
        dialogue = [
            {
                "role": "system",
                "content": "你是一个专业的总结专家。请参考示例风格，整合前面的解答。"
            },
            {
                "role": "user",
                "content": f"""示例：
    解答过程：
    问题：什么是量子计算？
    答案：量子计算利用量子力学原理进行计算。

    问题：量子计算有什么优势？
    答案：可以并行处理大量数据。

    总结：量子计算是一种基于量子力学原理的新型计算方式，其最大优势在于并行处理能力。

    现在请总结：
    解答过程：
    {context}

    原始问题：{complex_question}"""
            }
        ]
        
        prompt = apply_chat_template(dialogue)
        final_answer = generate_response(
            self.model,
            self.tokenizer,
            prompt,
            max_new_tokens=1536,
            temperature=0.5
        )
        
        return {
            "original_question": complex_question,
            "sub_solutions": solutions,
            "final_answer": final_answer
        }

In [5]:
# 使用示例部分的修改
if __name__ == "__main__":
    # 初始化模型和tokenizer
    model_path = "google/gemma-2-9b"
    cache_dir = "/root/autodl-tmp/gemma"
    # lora_path = "/root/autodl-tmp/models/stage1/checkpoints/gemma-base-zh/checkpoint-20000"
    lora_path = None

    model, tokenizer = initialize_model_and_tokenizer(
        model_path=model_path,
        cache_dir=cache_dir,
        lora_path=lora_path,
        use_quantization=True  # 确保使用量化
    )
    
    # 测试示例
    solver = L2MSolver(model, tokenizer)
    question = "在NLP领域，请介绍一下位置编码"
    result = solver.solve(question)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

/root/miniconda3/envs/Gemma/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The 'batch_size' attribute of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.
Token indices sequence length is longer than the specified maximum sequence length for this model (4624 > 4096). Running this sequence through the model will result in indexing errors
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (8192). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


In [6]:
print(result)

{'original_question': '在NLP领域，请介绍一下位置编码', 'sub_solutions': [{'question': '"""', 'solution': '""".format(question)\n     # 用户输入开始\nwhile True:\n   answer = input("请输入你的答案:")\n   print(\'你的答案:\', answer)\n   if (answer == \'exit\'):\n      break; \n  # 用户输入结束\n # 输出结果\n print(\'\\n\\n输出结果:\')\n for i in range(len(answers)):\n   print(answers[i][0]+\':\', answers[i][1])\n\n\n```\n\nIn []:\n## 模型部署及推理预测\n\nIn []:\n### 将模型保存为`pickle`格式文件\n\n将训练好的ChatGPT-LLM模型保存为`chatgpt_llm.pkl` pickle格式的文件，供用户调用使用。\n\nIn []:\n```python\nimport joblib\njoblib.dump(llm, "chatgpt_llm.pkl")\n\n```\n\nOut []:\n```output\n[\'chatgpt_llm.pkl\']\n```\n\nIn []:\n### 推理测试\n\n对一个随机的句子进行推理，查看结果是否与预期一致。\n\nIn []:\n```python\ndef predict(text):\n    return llm.generate(prompt=text, num_beams=5, max_length=20)\n\nrandom_sentence = "这是我第一次用LLM生成文本。"\nresult = predict(random_sentence)\nprint(result)\n\n```\n\nOut []:\n```output\n<output truncated>\n```\n\nIn []:\n### 使用API接口\n\n在本地服务器上创建一个应用，提供RESTful API接口供外部程序调用。\n\nIn [

In [7]:
result.keys()

dict_keys(['original_question', 'sub_solutions', 'final_answer'])

In [9]:
result["original_question"], result["final_answer"]

('在NLP领域，请介绍一下位置编码',
 '```\n\nIn []:\n```python\nfrom langchain.agents import ConversationAgent\nfrom langchain.agents import initialize_agent\nfrom langchain.agents.agent_types import AgentType\nfrom langchain.llms import OpenAI\n\nconversation_agent = ConversationAgent(\n    agent=initialize_agent(\n        llm=OpenAI(),\n        tools=[],\n        verbose=True,\n        verbose_level=2,\n        agent=ConversationAgent,\n        tool_templates={},\n        agent_type=AgentType.CHATBOT,\n    ),\n    verbose=True,\n    verbose_level=2,\n)\n\n```\n\nIn []:\n```python\nfrom langchain.agents import LLAMAAgent\nfrom langchain.prompts import PromptTemplates\nfrom langchain.llms import OpenAI\nfrom langchain.agents.agent_types import AgentType\n\ntemplates = ChainPromptTemplates(\n    chat_prompt="You will act as a helpful agent who helps users by answering their questions about sports."\n)\n\nlla = LLAMAAgent.from_chain_type(\n    LLAMAAgent.from_llm(\n        openai.OpenAI(temperature=0.7

In [10]:
result["sub_solutions"]

[{'question': '"""',
  'solution': '""".format(question)\n     # 用户输入开始\nwhile True:\n   answer = input("请输入你的答案:")\n   print(\'你的答案:\', answer)\n   if (answer == \'exit\'):\n      break; \n  # 用户输入结束\n # 输出结果\n print(\'\\n\\n输出结果:\')\n for i in range(len(answers)):\n   print(answers[i][0]+\':\', answers[i][1])\n\n\n```\n\nIn []:\n## 模型部署及推理预测\n\nIn []:\n### 将模型保存为`pickle`格式文件\n\n将训练好的ChatGPT-LLM模型保存为`chatgpt_llm.pkl` pickle格式的文件，供用户调用使用。\n\nIn []:\n```python\nimport joblib\njoblib.dump(llm, "chatgpt_llm.pkl")\n\n```\n\nOut []:\n```output\n[\'chatgpt_llm.pkl\']\n```\n\nIn []:\n### 推理测试\n\n对一个随机的句子进行推理，查看结果是否与预期一致。\n\nIn []:\n```python\ndef predict(text):\n    return llm.generate(prompt=text, num_beams=5, max_length=20)\n\nrandom_sentence = "这是我第一次用LLM生成文本。"\nresult = predict(random_sentence)\nprint(result)\n\n```\n\nOut []:\n```output\n<output truncated>\n```\n\nIn []:\n### 使用API接口\n\n在本地服务器上创建一个应用，提供RESTful API接口供外部程序调用。\n\nIn []:\n```python\nfrom flask import Flask, request, jsonify\